### **PydanticOuputParser**

`PydanticOuputParser`는 모델의 출력을 더 구조화된 정보로 변환 

- 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공 
- "필요 정보만! 출력"

주요 메서드 

- `get_format_instructions()`: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction) 제공. 
    - 모델이 출력해야 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환 

- `parse()`: 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환. 
    - Pydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환.


#### 참고 자료
- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)

In [1]:
from langchain_groq import ChatGroq 
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatGroq(model = 'gemma2-9b-it')

In [143]:
# 이메일 예시 문장
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
To: 이은채 (eunchae@teddyinternational.me)

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


### **출력 파서를 사용하지 않을때**


- 구조화된 결과를 출력하기 힘듬... 

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template('''입력된 텍스트는 이메일 입니다 
                                       
                                      
                                      {email_conversation}
                                      
                                      보낸사람, 받는사람, 메일 제목, 본문 요약 텍스트, 미팅 날짜와 시간
                                      만 추출해서 딕셔너리로 만들고싶어
                                      
                                      ex)
                                      {{sender: 김철수, receiver: 이은채.... meeting_data : 1월 15일 오전 10시}}
                                      
                                      딕셔너리로 된 최종 출력 결과만 제공해주세요.
                                      ''')

chain = prompt | llm

response = chain.invoke(email_conversation)

print(response.content)

```json
{
"sender": "김철수",
"receiver": "이은채",
"subject": "ZENESIS 자전거 유통 협력 및 미팅 일정 제안",
"summary": "바이크코퍼레이션 김철수 상무가 ZENESIS 자전거 브로슈어 요청 및 1월 15일 오전 10시 미팅 제안",
"meeting_data": "1월 15일 오전 10시"
}
```


### **Pydantic 사용**

In [144]:
# 필요한 단어 추출하기 위해 EmailSummary 클래스 생성
class EmailSummary(BaseModel):
    sender: str = Field(description="보낸 사람")
    mail : str = Field(description="보낸 사람 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일에 언급된 미팅 날짜와 시간")
    
    
# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# instruction 을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender": {"description": "보낸 사람", "title": "Sender", "type": "string"}, "mail": {"description": "보낸 사람 이메일 주소", "title": "Mail", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["sender", "mail", "subject", "summary", "date"]}
```


In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# 여러개의 변수(question, email_conversation, format) 중 하나에만 포맷팅
prompt = prompt.partial(format=parser.get_format_instructions())

### **1. Prompt | llm 만 Chain**

In [154]:
# chain 을 생성
chain = prompt | llm

# chain 을 실행하고 결과를 출력
# 간단한 질문 만으로 필요한 포맷대로 출력 가능함
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

output = ''
for content in response:
    print(content.content, end="", flush = True)
    output += content.content

```json
{
  "sender": "김철수",
  "mail": "chulsoo.kim@bikecorporation.me",
  "subject": "ZENESIS 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션 김철수 상무는 귀사의 신규 자전거 'ZENESIS'에 관심을 가지고 상세한 브로슈어를 요청하며 유통 전략 및 마케팅 계획 논의를 위한 다음 주 화요일 오전 10시 미팅을 제안합니다.",
  "date": "1월 15일 오전 10시"
}
```

### **정규표현식으로 원하는 부분 추출**

In [175]:
import re 
import json

# 🔹 JSON 부분만 정규식으로 추출
match = re.search(r'```json\n(.*?)\n```', output, re.DOTALL)   # 정규식으로 JSON 부분 추출 (.*?) : 임의의 문자열을 의미
json_str = match.group(1)  # JSON 부분만 추출
data = json.loads(json_str)  # JSON 문자열을 Python 딕셔너리로 변환
data

{'sender': '김철수',
 'mail': 'chulsoo.kim@bikecorporation.me',
 'subject': 'ZENESIS 자전거 유통 협력 및 미팅 일정 제안',
 'summary': "바이크코퍼레이션 김철수 상무는 귀사의 신규 자전거 'ZENESIS'에 관심을 가지고 상세한 브로슈어를 요청하며 유통 전략 및 마케팅 계획 논의를 위한 다음 주 화요일 오전 10시 미팅을 제안합니다.",
 'date': '1월 15일 오전 10시'}

### **parser 함수로 추출**

- 정규표현식보다 간단하게 추출 가능

In [186]:
struct_output = parser.parse(output)
struct_output

EmailSummary(sender='김철수', mail='chulsoo.kim@bikecorporation.me', subject='ZENESIS 자전거 유통 협력 및 미팅 일정 제안', summary="바이크코퍼레이션 김철수 상무는 귀사의 신규 자전거 'ZENESIS'에 관심을 가지고 상세한 브로슈어를 요청하며 유통 전략 및 마케팅 계획 논의를 위한 다음 주 화요일 오전 10시 미팅을 제안합니다.", date='1월 15일 오전 10시')

In [187]:
dict(struct_output)

{'sender': '김철수',
 'mail': 'chulsoo.kim@bikecorporation.me',
 'subject': 'ZENESIS 자전거 유통 협력 및 미팅 일정 제안',
 'summary': "바이크코퍼레이션 김철수 상무는 귀사의 신규 자전거 'ZENESIS'에 관심을 가지고 상세한 브로슈어를 요청하며 유통 전략 및 마케팅 계획 논의를 위한 다음 주 화요일 오전 10시 미팅을 제안합니다.",
 'date': '1월 15일 오전 10시'}

### **2. parser 가 추가된 체인 생성**

출력 결과를 정의한 Pydantic 객체로 생성

In [188]:
# 출력 파서를 추가하여 전체 체인을 재구성
chain = prompt | llm | parser

# chain 을 실행하고 결과를 출력
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력
# parser.parse() 와 동일한 결과
response

EmailSummary(sender='김철수', mail='chulsoo.kim@bikecorporation.me', subject='ZENESIS 자전거 유통 협력 및 미팅 일정 제안', summary="바이크코퍼레이션은 신규 자전거 'ZENESIS'에 관심을 가지며, 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청합니다. 1월 15일 화요일 오전 10시에 미팅을 제안하여 협력 가능성을 논의하고자 합니다.", date='1월 15일 화요일 오전 10시')

### **3. with_structured_output()**

- 모델 자체에 `parser` 추가

- `.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환 가능

- `.with_structured_output()` 함수는 `stream()` 기능을 지원하지 않음

In [196]:
llm_with_structered = ChatGroq(
    temperature=0, model_name="gemma2-9b-it").with_structured_output(EmailSummary)

response = llm_with_structered.invoke(email_conversation)
response

EmailSummary(sender='김철수', mail='chulsoo.kim@bikecorporation.me', subject='ZENESIS 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션 김철수 상무가 ZENESIS 자전거 유통 협력 및 미팅 일정을 제안하는 이메일입니다. ZENESIS 모델에 대한 상세한 브로슈어를 요청하고, 1월 15일 오전 10시에 미팅을 제안합니다.', date='2024-01-15')